In [1]:
%pip install wandb datasets transformers trl torch peft

In [2]:
import wandb
from transformers import AutoModelForCausalLM, AutoTokenizer, DataCollatorForLanguageModeling
from trl import SFTTrainer, SFTConfig
import torch
from peft import LoraConfig, get_peft_model

In [3]:
# !huggingface-cli login

In [4]:
# 모델과 토크나이저 로드
model_name = "gpt2-medium"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [5]:
import json
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset

# corpus.json 파일 읽기
with open('./data/corpus.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# train/test 분리 (80:20 비율)
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

print(f"전체 데이터 크기: {len(data)}")
print(f"학습 데이터 크기: {len(train_data)}")
print(f"테스트 데이터 크기: {len(test_data)}")

# 데이터 예시 출력
print("\n=== 학습 데이터 예시 ===")
print(train_data[0])
print("\n=== 테스트 데이터 예시 ===")
print(test_data[0])

# 리스트를 Dataset 형식으로 변환
train_dataset = Dataset.from_list(train_data)
test_dataset = Dataset.from_list(test_data)

전체 데이터 크기: 125
학습 데이터 크기: 100
테스트 데이터 크기: 25

=== 학습 데이터 예시 ===
{'input': '감정: 혼란스러움 / 원하는 효과: 위로받고 싶다 / 직업: 구직자 / 현재 상황: 스트레스 해소 / 집중도: 빠르게 몰입할 수 있는 책', 'instruction': '혼란스러운 심리 상태에서 위안을 얻고 싶을 때 적절한 3권을 골라, 추천 이유와 효과를 적으세요.', 'output': '1. 마음은 괜찮냐고 시가 물었다 - 황인환\n\n* 추천 이유: 시를 통해 감정의 혼란을 천천히 정돈할 수 있습니다.\n* 이 책이 도움이 될 수 있는 이유: 짧은 글이 매일의 위로가 되어줄 수 있습니다.\n\n2. 오렌지와 빵칼 - 청예\n\n* 추천 이유: 다양한 인물의 시선을 따라가며 자신과 상황을 분리해 생각할 수 있습니다.\n* 이 책이 도움이 될 수 있는 이유: 쉽게 몰입할 수 있는 이야기로 감정적 환기를 제공합니다.\n\n3. 이렇게 누워만 있어도 괜찮을까 - 안예슬\n\n* 추천 이유: 혼란스러운 변화 속에서 자신에게 더 관대해지는 법을 일깨워 줍니다.\n* 이 책이 도움이 될 수 있는 이유: 편안한 휴식의 순간을 통해 스스로 회복력을 느낄 수 있습니다.'}

=== 테스트 데이터 예시 ===
{'instruction': '슬럼프에 빠진 운동선수나 직장인에게 자기 극복과 성장 동기를 줄 책을 추천해줘.', 'input': '감정: 슬럼프, 무기력\n직업: 운동선수, 직장인\n효과: 자기 극복, 동기부여\n상황: 경기 또는 업무 후\n집중도: 몰입감 높은 책', 'output': '1. 운동의 진실과 기쁨 - 강윤규 지음\n   - 추천 이유: 운동을 통한 자기성찰과 회복의 과정이 담긴 책입니다.\n   - 이 책이 도움이 될 수 있는 이유: 운동의 본질적 기쁨과 극복 경험을 통해 재충전할 수 있습니다.\n2. 해나가는 힘 : 늘 사랑받고 싶은 사람을 위한 심리학 - 이승현\n   - 추천 이유: 자신을 사랑하고 자신감과 추진력

In [6]:
# 프롬프트 포매팅 함수 정의
def formatting_func(example):
    # instruction, input, output이 단일 문자열인 경우를 처리
    instruction = example['instruction']
    input_text = example['input'].strip() if example['input'] else ""
    output = example['output']

    # 형식화된 프롬프트 생성
    text = f"[Instruction]\n{instruction.strip()}\n\n"
    if input_text:
        text += f"[Input]\n{input_text}\n\n"
    text += f"[Output]\n{output.strip()}"

    return text

# 데이터 콜레이터 설정
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # Causal Language Modeling을 위해 False로 설정
)

In [7]:
# import gc

# gc.collect()
# torch.cuda.empty_cache()

# wandb.init(
#     project='Hanghae99-book-recommandation',
#     name=f'gpt-finetuning',
#     reinit=True
# )

# model = AutoModelForCausalLM.from_pretrained(
#     model_name,
# )

# trainer = SFTTrainer(
#     model,
#     train_dataset=train_dataset,
#     eval_dataset=test_dataset,
#     args=SFTConfig(
#         output_dir=f"/tmp/clm-instruction-tuning",
#         max_seq_length=128,
#         eval_strategy="epoch",
#         save_strategy="epoch",
#         logging_steps=10,
#         num_train_epochs=3,
#         learning_rate=2e-5,
#         per_device_train_batch_size=1,
#         gradient_accumulation_steps=4,
#         load_best_model_at_end=True,
#         metric_for_best_model="eval_loss"
#     ),
#     formatting_func=formatting_func,
#     data_collator=data_collator,
# )

# # 학습 시작
# train_result = trainer.train()
# metrics = train_result.metrics
# trainer.log_metrics("train", metrics)

# trainer.save_model()

# wandb.finish()

### LoRA 적용

In [17]:
def train_with_lora(lora_r, model_name, train_dataset, test_dataset, formatting_func, data_collator):
    # 디바이스 자동 선택: CUDA > MPS > CPU
    if torch.cuda.is_available():
        device = torch.device("cuda")
        torch_dtype = torch.float16  # CUDA에서는 float16이 빠르고 효율적
        use_fp16 = True
        device_map = 'auto'
        print("CUDA를 사용합니다.")
    elif getattr(torch.backends, 'mps', None) and torch.backends.mps.is_available():
        device = torch.device("mps")
        torch_dtype = torch.float32  # MPS에서는 float32가 안전함
        use_fp16 = False
        device_map = None
        print("MPS를 사용합니다.")
    else:
        device = torch.device("cpu")
        torch_dtype = torch.float32
        use_fp16 = False
        device_map = None
        print("CPU를 사용합니다.")

    wandb.init(
        project='Hanghae99-book-recommandation',
        name=f'{model_name}-with-lora-r{lora_r}',
        reinit=True
    )

    print(f"\n=== Training with LoRA rank {lora_r} ===")

    # 모델 로드 (device_map이 있을 때만 넣음)
    model_kwargs = {
        "torch_dtype": torch_dtype
    }
    if device_map is not None:
        model_kwargs["device_map"] = device_map
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        **model_kwargs
    )

    # device_map이 없으면 명시적으로 모델 이동
    if device_map is None:
        model = model.to(device)

    # LoRA 설정
    lora_config = LoraConfig(
        r=lora_r,
        lora_alpha=32,
        target_modules=["q_proj", "k_proj", "v_proj", "out_proj", "fc1", "fc2"],
        lora_dropout=0.1,
        bias="none",
        task_type="CAUSAL_LM"
    )
    model = get_peft_model(model, lora_config)
    model.print_trainable_parameters()

    # SFTTrainer 정의
    trainer = SFTTrainer(
        model,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        args=SFTConfig(
            output_dir=f"/tmp/clm-instruction-tuning-lora-{lora_r}",
            max_seq_length=128,
            eval_strategy="epoch",
            save_strategy="epoch",
            logging_steps=10,
            num_train_epochs=3,
            learning_rate=5e-5,
            load_best_model_at_end=True,
            metric_for_best_model="eval_loss",
            per_device_train_batch_size=1,  # MPS/CPU 환경은 1, CUDA는 4까지도 가능(필요시 조건 분기)
            gradient_accumulation_steps=4,
            fp16=use_fp16
        ),
        formatting_func=formatting_func,
        data_collator=data_collator,
    )

    # 모델이 올바른 device로 있는지 추가 확인 필요 시 trainer.model = trainer.model.to(device)
    if device_map is None:
        trainer.model = trainer.model.to_empty(device)

    # 학습 시작
    train_result = trainer.train()
    metrics = train_result.metrics
    trainer.log_metrics("train", metrics)

    if torch.cuda.is_available():
        print('Max Alloc:', round(torch.cuda.max_memory_allocated(0)/1024**3, 1), 'GB')

    # 모델 저장
    trainer.save_model()

    wandb.finish()

In [18]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) 
Token is valid (permission: read).

In [1]:
train_with_lora(
    lora_r=8,
    model_name="google/gemma-7b-it",
    train_dataset=train_dataset,
    test_dataset=test_dataset,
    formatting_func=formatting_func,
    data_collator=data_collator
)

NameError: name 'train_with_lora' is not defined